In [ ]:
import json
from tqdm import tqdm
import requests

In [ ]:
# Load Data
with open('data-collection\data\chapter-data.json', 'r') as f:
    data = json.load(f)

In [ ]:
url = "https://api-inference.huggingface.co/models/meta-llama/Llama-3.2-3B-Instruct"
token = ""  # Replace with your Hugging Face token

def model(prompt):
    headers = {
      'Authorization': f'Bearer {token}'
    }

    parameters = {
      "max_new_tokens": 100,
      "temperature": 0.01,
      "top_k": 50,
      "top_p": 0.95,
      "return_full_text": False
    }

    payload = {"inputs": prompt, "parameters": parameters}

    response = requests.post(url, headers=headers, json=payload)
    
    if response.status_code == 200:
        print("?", response.json())
        return response.json()[0]['generated_text']
    else:
        print(f'Error: {response.status_code}')
        print(response.json())
        return None

In [ ]:
dataset = []

# Loop through each excerpt
for entry in tqdm(data):
    book_title = entry.get("book_title", "")
    chapter_name = entry.get("chapter_name", "")
    paragraphs = entry.get("paragraphs", [])

    for paragraph in paragraphs:
        if len(paragraph) < 100:
            continue

        input_data = f"Book Title: {book_title}\nChapter Name: {chapter_name}\n\n{paragraph}"

        # Generate question 
        question_prompt = f"Generate one reflective or thoughtful question based on the following passage:\n\n{paragraph}"

        response_question = model(question_prompt)

        question = response_question.choices[0].message.content.strip()


        # Generate answer from question
        answer_prompt = (
            f"Answer the following question in the style of the historical figure who wrote the text below:\n\n"
            f"Text: {paragraph}\n\nQuestion: {question}\n\nAnswer:"
        )

        response_answer = model(answer_prompt)
        print(response_answer)

        answer = response_answer.choices[0].message.content.strip()


        dataset.append({
            "question": question,
            "input": input_data,
            "answer": answer
        })

    if len(dataset) >= 10:
        break



In [ ]:

# Save the formatted instruction-tuning data
with open('instruction_tuning_dataset.jsonl', 'w') as f:
    for item in instruction_dataset:
        f.write(json.dumps(item) + "\n")